In [1]:
# magics
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Imports

In [2]:
# Regular Imports
import os, sys, configparser
import time
from collections import OrderedDict

In [3]:
# Scientific imports
from joblib import Memory

import dask
from dask import delayed
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SGECluster
from dask.distributed import fire_and_forget

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
# Dask functions
from functions import (fit, 
                       get_reference_map, 
                       load_sample, 
                       evaluate_model, 
                       cluster_outliers, 
                       filter_clusters, 
                       estimate_bdcs, 
                       make_event_map, 
                       make_shell_maps, 
                       make_event_table,
                       merge_event_tables,
                       output_event_table,
                      )

from scheduler import (get_client,
                       replicate_dict_futures,
                      )

from pandda_analyse.criticise import (make_z_map,
                                      make_event_map,
                                      make_mean_map,
                                     )

In [5]:
# Multi Dataset Crystalography imports
import multi_dataset_crystalography as mdc # import MultiCrystalDataset
from multi_dataset_crystalography.utils import DefaultPanDDADataloader
from multi_dataset_crystalography.dataset.sample_loader import PanddaDiffractionDataTruncater



In [6]:
# PanDDA imports
from pandda_analyse.config import PanDDAConfig
from pandda_analyse.event_model import PanDDAEventModel
# from pandda_analyse.processor import ProcessModelSeriel
from pandda_analyse.event_model_distributed import PanDDAEventModelDistributed
from pandda_analyse.pandda_phil import pandda_phil
from pandda_config import extract_params_default
from pandda_analyse.pandda_output import (Dir,
                                         File,
                                          IndexedFile,
                                         )


In [7]:
# functional prpgramming
from functional import (chain, 
                        curry_kwargs,
                       )

# Args

In [8]:
# Arguments
arguments = None
# Config
config_path = "/dls/science/groups/i04-1/conor_dev/pandda/lib-python/pandda/pandda_analyse/pandda_analyse/analyse_config.ini"
config = configparser.ConfigParser()


# Config

In [9]:
# PDK2
# args = ["data_dirs=/dls/labxchem/data/2018/lb19758-9/processing/analysis/initial_model/*",
#        "out_dir=/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir_2",
#        "pdb_style=dimple.pdb",
#        "mtz_style=dimple.mtz",
#        "cpus=21",
#        ]
# XXKARNA
args = ["data_dirs=/dls/labxchem/data/2017/lb18145-17/processing/analysis/initial_model/*",
       "out_dir=/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir_2",
       "pdb_style=dimple.pdb",
       "mtz_style=dimple.mtz",
       "cpus=21",
       ]


In [10]:
pandda_start_time = time.time()

working_phil = extract_params_default(master_phil=pandda_phil,
                                      args=args,
                                      blank_arg_prepend=None,
                                      home_scope=None
                                     ).extract()

# Maps options to code abstractions
pandda_config = PanDDAConfig(working_phil)

In [11]:
# Get Dataset
pandda_dataset = mdc.dataset.dataset.MultiCrystalDataset(dataloader=pandda_config.dataloader,
                                                         sample_loader=pandda_config.sample_loader,
                                                         )

In [12]:
# Get reference
reference = pandda_config.get_reference(pandda_dataset.datasets)
pandda_dataset.sample_loader.reference = reference

In [13]:
# transform dataset
dataset = chain(pandda_dataset, [curry_kwargs(pandda_config.transform_data_check,
                                              reference=reference,
                                             ),
                                 curry_kwargs(pandda_config.transform_scale_diffraction, 
                                              reference_dataset=reference,
                                             ),
                                 curry_kwargs(pandda_config.transform_filter_structure,
                                              reference_dataset=reference,
                                             ),
                                 curry_kwargs(pandda_config.transform_filter_wilson,
                                              reference=reference,
                                             ),
                                 curry_kwargs(pandda_config.transform_align, 
                                              reference_dataset=reference,
                                             )
                                ]
               )

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/zoh22914/.local/lib/python2.7/site-packages/panddas-0.2.12-py2.7.egg/bamboo/stats/__init__.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return 0.6745*devs/mdev
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/multi_dataset_crystalography/dataset/transforms.py:337: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_all_z') > self.max_wilson_plot_z_score) or \
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/multi_dataset_crystalography/dataset/transforms.py:339: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_<4A_z') >

[Parallel(n_jobs=21)]: Done 101 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 102 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 103 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 104 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 105 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 106 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 107 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 108 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 109 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 110 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 111 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 112 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 113 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 114 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 115 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 116 tasks      | elapsed:  1.0min
[Paralle

[Parallel(n_jobs=21)]: Done 237 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 238 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 239 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 240 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 241 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 242 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 243 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 244 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 245 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 246 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 247 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 248 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 249 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 250 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 251 tasks      | elapsed:  1.2min
[Parallel(n_jobs=21)]: Done 252 tasks      | elapsed:  1.2min
[Paralle

[Parallel(n_jobs=21)]: Done 372 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 373 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 374 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 375 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 376 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 377 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 378 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 379 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 380 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 381 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 382 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 383 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 384 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 385 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 386 tasks      | elapsed:  1.3min
[Parallel(n_jobs=21)]: Done 387 tasks      | elapsed:  1.3min
[Paralle

[Parallel(n_jobs=21)]: Done 507 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 508 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 509 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 510 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 511 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 512 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 513 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 514 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 515 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 516 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 517 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 518 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 519 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 520 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 521 tasks      | elapsed:  1.4min
[Parallel(n_jobs=21)]: Done 522 tasks      | elapsed:  1.4min
[Paralle

...

[Parallel(n_jobs=21)]: Done 779 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 780 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 781 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 782 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 783 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 784 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 785 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 786 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 787 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 788 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 789 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 790 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 791 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 792 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 793 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 794 tasks      | elapsed:  1.7min
[Paralle

[Parallel(n_jobs=21)]: Done 915 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 916 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 917 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 918 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 919 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 920 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 921 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 922 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 923 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 924 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 925 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 926 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 927 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 928 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 929 tasks      | elapsed:  1.8min
[Parallel(n_jobs=21)]: Done 930 tasks      | elapsed:  1.8min
[Paralle

[Parallel(n_jobs=21)]: Done 1049 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1050 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1051 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1052 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1053 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1054 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1055 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1056 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1057 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1058 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1059 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1060 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1061 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1062 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1063 tasks      | elapsed:  2.0min
[Parallel(n_jobs=21)]: Done 1064 tasks      | elapsed: 

[Parallel(n_jobs=21)]: Done 1181 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1182 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1183 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1184 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1185 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1186 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1187 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1188 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1189 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1190 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1191 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1192 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1193 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1194 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1195 tasks      | elapsed:  2.1min
[Parallel(n_jobs=21)]: Done 1196 tasks      | elapsed: 

[Parallel(n_jobs=21)]: Done 1315 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1316 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1317 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1318 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1319 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1320 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1321 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1322 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1323 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1324 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1325 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1326 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1327 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1328 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1329 tasks      | elapsed:  2.2min
[Parallel(n_jobs=21)]: Done 1330 tasks      | elapsed: 

In [14]:
# Get grid
grid = pandda_config.grid_loader(reference)

----------------------------------->>>
Atomic Mask Summary:
Total Mask Size (1D): 578016
Outer Mask Size (1D): 866531
Inner Mask Size (1D): 288515
Masked Grid Min/Max: ((9, 63, 63), (139, 65, 93))
----------------------------------->>>
Atomic Mask Summary:
Total Mask Size (1D): 794730
Outer Mask Size (1D): 2441710
Inner Mask Size (1D): 1646980
Masked Grid Min/Max: ((0, 0, 0), (146, 132, 166))


In [15]:
# Define event Model
pandda_event_model = PanDDAEventModel(pandda_config.statistical_model,
                                      pandda_config.clusterer,
                                      pandda_config.event_finder,
                                      bdc_calculator=pandda_config.bdc_calculator,
                                      statistics=[],
                                      map_maker=pandda_config.map_maker,
                                      event_table_maker=pandda_config.event_table_maker,
                                      cpus=pandda_config.config.settings.cpus,
                                      tree=None,
                                     )

In [16]:
# Get partitions
dataset.partitions = pandda_config.partitioner(dataset.datasets)

In [17]:
# instatiate a dataloader for the datasets
dataloader = DefaultPanDDADataloader(min_train_datasets=60,
                                     max_test_datasets=100,
                                    )

In [18]:
# Get the datasets to iterate over
datasets = {idx: d 
            for idx, d 
            in dataloader(dataset)
           }

Got all train datasets
Sorted datasets
Collected trian datasets
Got all test datasets
sorted test datasets
yielding dataset
Dataset 0 of length 100; res limits (1.62986514679,1.67955090398)
yielding dataset
Dataset 1 of length 160; res limits (1.67957882186,1.74986735839)
yielding dataset
Dataset 2 of length 160; res limits (1.74988205719,1.80010261078)
yielding dataset
Dataset 3 of length 160; res limits (1.80014427644,1.8499774446)
yielding dataset
Dataset 4 of length 160; res limits (1.84998557751,1.8941647794)
yielding dataset
Dataset 5 of length 160; res limits (1.89425766029,1.9307579843)
yielding dataset
Dataset 6 of length 160; res limits (1.93151889617,1.97982872565)
yielding dataset
Dataset 7 of length 160; res limits (1.97985015777,2.01983757916)
yielding dataset
Dataset 8 of length 160; res limits (2.01984674168,2.07005723959)
yielding dataset
Dataset 9 of length 160; res limits (2.07014107371,2.12021842588)
yielding dataset
Dataset 10 of length 160; res limits (2.120293301

In [19]:
# Get file tree
tree = Dir(name="pandda_test_9",
          root=pandda_config.config.output.out_dir,
          children={"datasets": Dir(name="datasets",
                                   children={dtag: Dir(name=dtag,
                                                       children={"z_map": File(name="zmap.ccp4"),
                                                                 "event_map": IndexedFile(name="event_map_{}.ccp4"),
                                                               }
                                                      )
                                            for dtag, dts
                                            in dataset.datasets.items()
                                            },
                                   ),
                   "shells": Dir(name="shells",
                               children={shell: Dir(name=str(shell),
                                                   children={"mean_map": File(name="mean_map.ccp4"),
                                                            "event_table": File(name="event_table.csv"),
                                                            }
                                                   )
                                        for shell, dts
                                        in datasets.items()
                                        }
                               ),
                    "event_table": File(name="event_table.csv"),
                    "analyses": Dir(name="analyses",
                                   children={"pandda_analyse_events": File(name="pandda_analyse_events.csv")},
                                   )
                   }
          )

In [20]:
# Make the file system
tree.make()

In [25]:
# Iterate over resolution shells

futures = OrderedDict()
event_tables_computed = OrderedDict()
for shell_num, shell_dataset in datasets.items():
    futures[shell_num] = OrderedDict()
    
#     if shell_num > 3:
#         break
    
    shell_start = time.time()
    
    # ###############################################
    # Get client
    # ###############################################

    try: 
        print(client.cluster.dashboard_link)

    except Exception as e:
        print(e)
        dask.config.set({"distributed.admin.tick.limit": "300s"})
        
        # cluster= pandda_config.cluster()

        cluster = SGECluster(queue="medium.q",
                             project="labxchem",
                             cores=10,
                             processes=5,
                             memory="64GB",
                             resource_spec="m_mem_free=64G,redhat_release=rhel7",
                             python="/dls/science/groups/i04-1/conor_dev/ccp4/build/bin/cctbx.python",
                             walltime="06:00:00",
                             )
        cluster.scale(100)

        time.sleep(30)

        client = Client(cluster)
        print(client.cluster.dashboard_link)

    # ###############################################
    # Get resolution
    # ###############################################
    resolutions_test = max([dts.data.summary.high_res 
                            for dtag, dts
                            in shell_dataset.partition_datasets("test").items()
                           ]
                          )
    resolutions_train = max([dts.data.summary.high_res 
                             for dtag, dts
                             in shell_dataset.partition_datasets("train").items()
                            ]
                           )
    max_res = max(resolutions_test, resolutions_train)

    # ###############################################
    # Instantiate sheel variable names
    # ###############################################

    # Dataset names
    dtags = set(shell_dataset.partition_datasets("test").keys()
                + shell_dataset.partition_datasets("train").keys()
                )

    train_dtags = [dtag
                   for dtag
                   in dtags
                   if (dtag in shell_dataset.partition_datasets("train").keys())
                  ]
    test_dtags = [dtag
                  for dtag
                  in dtags
                  if (dtag in shell_dataset.partition_datasets("test").keys())
                 ]

    # ###############################################
    # Truncate datasets
    # ###############################################
    # TODO: move to imports section

    truncated_reference, truncated_datasets = PanddaDiffractionDataTruncater()(shell_dataset.datasets,
                                                                               reference,
                                                                              )

    # record max res of shell datasets
    shell_max_res = max_res

    # ###############################################
    # Generate maps
    # ###############################################
    # Generate reference map for shell

#     grid_scattered = client.scatter(grid)
    futures[shell_num]["grid_scattered"] = client.scatter(grid)
    print("replicating grid")
    client.replicate(futures[shell_num]["grid_scattered"])
    print("replicated grid")

    futures[shell_num]["shell_ref_map"] = client.submit(get_reference_map,
                                  pandda_config.reference_map_getter,
                                  reference,
                                  shell_max_res,
                                  futures[shell_num]["grid_scattered"],
                                 )
    print("replicating shell map")
    client.replicate(futures[shell_num]["shell_ref_map"])
    print("replicated shell map")
    
    # Load maps
    futures[shell_num]["xmaps"] = {}
    for dtag in dtags:
        futures[shell_num]["xmaps"][dtag] = client.submit(load_sample,
                                    pandda_config.map_loader,
                                    truncated_datasets[dtag],
                                    futures[shell_num]["grid_scattered"],
                                    futures[shell_num]["shell_ref_map"],
                                    shell_max_res,
                                   )
    client.replicate([xmap 
                      for dtag, xmap 
                      in futures[shell_num]["xmaps"].items() # xmaps.items()
                     ],
                     n=5,
                    )
    
    # ###############################################
    # Fit statistical model to trianing sets
    # ###############################################
    xmaps_computed = {dtag: xmap.result()
                      for dtag, xmap
                      in futures[shell_num]["xmaps"].items()  # xmaps.items()
                     }
    
    print("replicating xmaps")

    futures[shell_num]["shell_fit_model"] = fit(pandda_config.statistical_model,
                          [xmaps_computed[dtag] for dtag in train_dtags],
                          [xmaps_computed[dtag] for dtag in test_dtags],
                          )

    futures[shell_num]["shell_fit_model_scattered"] = client.scatter(futures[shell_num]["shell_fit_model"])
    client.replicate(futures[shell_num]["shell_fit_model_scattered"])

    # ###############################################
    # Find events
    # ###############################################
    futures[shell_num]["zmaps"] = {}
    futures[shell_num]["clusters"] = {}
    futures[shell_num]["events"] = {}
    futures[shell_num]["bdcs"] = {}
    for dtag in test_dtags:
        # Get z maps by evaluating model on maps
        futures[shell_num]["zmaps"][dtag] = client.submit(evaluate_model,
                                    futures[shell_num]["shell_fit_model_scattered"],
                                    futures[shell_num]["xmaps"][dtag],
                                   )

        # Cluster outlying points in z maps
        futures[shell_num]["clusters"][dtag] = client.submit(cluster_outliers,
                                       pandda_config.clusterer,
                                       truncated_datasets[dtag],
                                       futures[shell_num]["zmaps"][dtag],
                                       futures[shell_num]["grid_scattered,
                                      )

        # Find events by filtering the clusters
        futures[shell_num]["events"][dtag] = client.submit(filter_clusters,
                                     pandda_config.event_finder,
                                     truncated_datasets[dtag],
                                     futures[shell_num]["clusters"][dtag],
                                     futures[shell_num]["grid_scattered"],
                                    )
    
    client.replicate([zmap 
                      for dtag, zmap 
                      in futures[shell_num]["zmaps"].items()
                     ],
                    n=3,
                    )
    client.replicate([cluster 
                      for dtag, cluster
                      in futures[shell_num]["clusters"].items()
                     ],
                    n=3,
                    )
    client.replicate([event 
                      for dtag, event 
                      in futures[shell_num]["events"].items()
                     ],
                    n=3,
                    )

    # Calculate background correction factors
#     for dtag in test_dtags:
#         bdcs[dtag] = client.submit(estimate_bdcs,
#                                     pandda_config.bdc_calculator,
#                                     truncated_datasets[dtag],
#                                     xmaps[dtag],
#                                     shell_ref_map,
#                                     events[dtag],
#                                     grid_scattered,
#                                    )
    futures[shell_num]["events_analysed"] = OrderedDict()
    for dtag in test_dtags:
        futures[shell_num]["events_analysed"][dtag] = client.submit(pandda_config.event_analyser,
                                              truncated_datasets[dtag],
                                              futures[shell_num]["xmaps"][dtag],
                                              futures[shell_num]["shell_ref_map"],
                                              futures[shell_num]["events"][dtag],
                                              futures[shell_num]["grid_scattered"],
                                   )
        
#     client.replicate([bdc 
#                     for dtag, bdc 
#                     in bdcs.items()
#                    ],
#                    n=3,
#                   )

    client.replicate([event_analysis 
                    for dtag, event_analysis 
                    in futures[shell_num]["events_analysed"].items()
                   ],
                   n=3,
                  )
    
#     bdcs_computed = {dtag: bdc.result()
#                     for dtag, bdc
#                     in bdcs.items()
#                     }
    events_analysed_computed = {dtag: events_analysis.result()
                    for dtag, events_analysis
                    in futures[shell_num]["events_analysed"].items()
                    }
    print(events_analysed_computed)
    
    
    events_computed = {dtag: event.result()
                       for dtag, event
                       in futures[shell_num]["events"].items()
                      }

    # Criticise each indiidual dataset (generate statistics, event map and event table)
    futures[shell_num]["z_maps_ccp4"] = OrderedDict()
#     for dtag in test_dtags:
#         z_maps_ccp4[dtag] = client.submit(make_z_map,
#                       xmaps[dtag], 
#                       truncated_datasets[dtag], 
#                       tree["datasets"][dtag]["z_map"](), 
#                       shell_fit_model_scattered, 
#                       grid_scattered,
#                      )
    
    futures[shell_num]["event_maps"] = OrderedDict()
    for dtag in test_dtags:
        event_maps[dtag] = {}
        for event_id, events_analysis_computed in events_analysed_computed[dtag].items():
            futures[shell_num]["event_maps"][dtag][event_id] = client.submit(make_event_map,
                                                       futures[shell_num]["xmaps"][dtag],
                                                       truncated_datasets[dtag], 
                                                       futures[shell_num]["shell_ref_map"], 
                                                       futures[shell_num]["events_computed"][dtag][2][int(event_id[1])-1],
                                                       events_analysed_computed[dtag][event_id]["estimated_bdc"],
                                                       tree["datasets"][dtag]["event_map"](event_id[1]),
                                                       futures[shell_num]["grid_scattered"],
                                                       )
        if len(events_computed[dtag][2]) != 0:
            futures[shell_num]["z_maps_ccp4"][dtag] = client.submit(make_z_map,
              futures[shell_num]["xmaps"][dtag], 
              truncated_datasets[dtag], 
              tree["datasets"][dtag]["z_map"](), 
              futures[shell_num]["shell_fit_model_scattered"], 
              futures[shell_num]["grid_scattered"],
             )

    print(tree["shells"][shell_num]["mean_map"]())
    futures[shell_num]["shell_maps"] = client.submit(make_mean_map,
                               reference,
                               futures[shell_num]["shell_ref_map"],
                               futures[shell_num]["grid_scattered"],
                               tree["shells"][shell_num]["mean_map"](),
                               )
    print("Making event map")

    futures[shell_num]["event_table"] = client.submit(make_event_table,
                                 pandda_config.event_table_maker,
                                 tree["shells"][shell_num]["event_table"](),
                                 shell_dataset,
                                 events_computed,
                                futures[shell_num]["grid_scattered"],
                                events_analysed_computed,
                                )
    print(event_table)

    event_maps_computed = OrderedDict()
    print("geting event maps")

#     shell_maps_computed = shell_maps.result()
    fire_and_forget(futures[shell_num]["shell_maps"])
    print("Fire and forget shell maps at {}".format(time.time() - shell_start))

    
    for dtag, z_map in futures[shell_num]["z_maps_ccp4"].items():
#         z_maps_computed = z_map.result()
        fire_and_forget(z_map)
    print("Fire and forget z maps at {}".format(time.time() - shell_start))

    
    for dtag, futures_dict in futures[shell_num]["event_maps"].items():
        for event_id, future in futures_dict.items():
#             event_maps_computed = future.result()
            fire_and_forget(future)
    print("Fire and forget event maps at {}".format(time.time() - shell_start))
        
    
    print("Getting event table")
    event_tables_computed[shell_num] = futures[shell_num]["event_table"].result()
    print("Got shell table at at {}".format(time.time() - shell_start))


    print("restarting client")
#     client.restart()
    print("Shell processed in: {}".format(time.time() - shell_start))
    time.sleep(10)

http://172.23.159.3:41112/status
replicating grid
replicated grid
replicating shell map
replicated shell map
replicating xmaps
	### Fitting mu!
>> 100 % [==================================================]
	### Fitting sigma_uncertainty!
1   5    10   15   20   25   30   35   40   45   50   55   60   65   70   75   80   85   90   95   100  
|                                                                                                   |
	### Fitting sigma_adjusted!
>> 100 % [==================================================]


ValueError: Inputs contain futures that were created by another client.

In [22]:
client.close()
del client

In [ ]:
joined_event_table = merge_event_tables(event_tables_computed)

In [ ]:
output_event_table(joined_event_table,
                  tree["analyses"]["pandda_analyse_events"](),
                  )

In [ ]:


del client





dir(client)
print(client.status)



















